In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('../src')


In [11]:
from scraper.scraper import scrape_range
base_path = "https://listado.mercadolibre.com.pe/ropa-accesorios/calzado/zapatillas"

df = scrape_range(base_path, 1, 20)


In [15]:
df.to_parquet("../data/raw/raw_data.parquet")

In [15]:
# Preprocessing
# Eliminar los que tengan descripción y offer_price en nulo
import pandas as pd
df = pd.read_parquet("../data/raw/raw_data.parquet")
def filter_null(df, field):
    return df[df[field].notnull()]

df = filter_null(df, 'description')
df = filter_null(df, 'offer_price')

def clean_price(price):
    """
    Limpia el string de precio para obtener solo el valor numérico.
    Maneja casos como 'S/279' o 'S/248,40' convirtiéndolos a 279.0 y 248.40 respectivamente.
    """
    if not isinstance(price, str) or pd.isna(price):
        return None
    
    # Eliminar el símbolo de moneda y espacios en blanco
    price = price.replace("S/", "").strip()
    
    # Si hay una coma, tratarla como separador decimal
    if "," in price:
        parts = price.split(",")
        price = f"{parts[0].strip()}.{parts[1].strip()}"
    
    # Eliminar cualquier carácter que no sea número o punto decimal
    price = "".join(char for char in price if char.isdigit() or char == ".")
    
    try:
        return float(price)
    except (ValueError, TypeError):
        return None


df["offer_price"] = df['offer_price'].apply(clean_price)
df.head()


,description,brand,original_price,offer_price
0,Aqua Shoes Sun Negro | Aquashoes Suela Gruesa ...,None,None,45.00
1,Zapatilla Skechers Uno Gen1-shimmer 155196/wht...,SKECHERS,None,279.00
2,Zapatillas Originales Hombre Nike Tiempo Legen...,NIKE,"S/\n289\n,\n90",173.94
3,Aqua Shoes Sun Negro | Aquashoes Suela Gruesa ...,None,None,45.00
4,Zapatilla Skechers Uno Gen1-shimmer 155196/wht...,SKECHERS,None,279.00
